In [1]:
from eitprocessing.mixins.slicing import SelectByIndex
from dataclasses import dataclass, is_dataclass
from eitprocessing.eit_data.vendor import Vendor
from eitprocessing.eit_data.draeger import DraegerEITData
from eitprocessing.eit_data.eit_data_variant import EITDataVariant
from typing_extensions import Self

import os
import pytest
from pprint import pprint
import bisect
import numpy as np


In [2]:

draeger_file1 = '/home/dbodor/git/EIT-ALIVE/eitprocessing/tests/test_data/Draeger_Test3.bin'
data = DraegerEITData.from_path(draeger_file1)

# pprint(data)
data.time


array([75628.471, 75628.521, 75628.571, ..., 75719.321, 75719.371,
       75719.421])

In [3]:
cutoff = 10
print(data.label)
print(data[cutoff].label)
print(data[cutoff:cutoff+1].label)
print(data[0:cutoff].label)
print(data[cutoff:0].label)
print(data[cutoff:cutoff+2].label)


DraegerEITData_140210416651520
Frame (10) of <DraegerEITData_140210416651520>
Frame (10) of <DraegerEITData_140210416651520>
Frames (0-9) of <DraegerEITData_140210416651520>
No frames selected from <DraegerEITData_140210416651520>
Frames (10-11) of <DraegerEITData_140210416651520>


In [10]:
print('Slicing at exact timepoints')
print('These should all be identical')

tp0 = 75628.471
tp2 = 75628.571

print(data.select_by_time(0,tp2).time)
print(data.select_by_time(0,tp2, start_inclusive=False).time)
print(data.select_by_time(0,tp2, end_inclusive=False).time)
print("")
print(data.select_by_time(tp0,tp2).time)
print(data.select_by_time(tp0,tp2, start_inclusive=False).time)
print(data.select_by_time(tp0,tp2, start_inclusive=False, end_inclusive=False).time)
print(data.select_by_time(tp0,tp2, start_inclusive=False, end_inclusive=True).time)


Slicing at exact timepoints
These should all be identical
[75628.471 75628.521]
[75628.471 75628.521]
[75628.471 75628.521]

[75628.471 75628.521]
[75628.471 75628.521]
[75628.471 75628.521]
[75628.471 75628.521 75628.571]


In [5]:
print('Slicing in between timepoints')
print('for each `False` should exclude be 1 shorter')

tp0_ = tp0 + 0.01
tp2_ = tp2 - 0.01

for start_incl in (True, False):
    for end_incl in (True, False):
        print(start_incl, end_incl, end=": ")
        print(data.select_by_time(tp0_,tp2_,  start_incl, end_incl).time)


Slicing in between timepoints
for each `False` should exclude be 1 shorter
True True: [75628.471 75628.521 75628.571]
True False: [75628.471 75628.521]
False True: [75628.521 75628.571]
False False: [75628.521]


In [11]:
print('Slicing before first or after last timepoint')
print('each set of 3 should be identical')
print('')

tp0__ = tp0 - 0.01
tp_2 = 75719.321
tpx__ = 75719.421 + 0.01

print(tp0)
print(f'slice before 1st timepoint {tp0__}')
print(data.time[:3])
print(data.select_by_time(tp0, tp2, start_inclusive=True).time)
print(data.select_by_time(tp0__, tp2, start_inclusive=False).time)

print('')

print(f'slice after last timepoint: {tpx__}')
print(data.time[-3:])
print(data.select_by_time(tp_2, tpx__, end_inclusive=True).time)
print(data.select_by_time(tp_2, tpx__, end_inclusive=False).time)


Slicing before first or after last timepoint
each set of 3 should be identical

75628.471
slice before 1st timepoint 75628.46100000001
[75628.471 75628.521 75628.571]
[75628.471 75628.521]
[75628.471 75628.521]

slice after last timepoint: 75719.431
[75719.321 75719.371 75719.421]
[75719.321 75719.371 75719.421]
[75719.321 75719.371 75719.421]


In [7]:
data[1].time


array([75628.521])

In [14]:
data.select_by_time(75628.521, 75628.521, end_inclusive=True).time


array([75628.521])

In [9]:
tp1_ = data.time[1] - 0.01
tp4_ = data.time[3] - 0.01

print(data.time[1])
print(data.time[3])
data.select_by_time(tp1_, tp4_).time


75628.521
75628.621


array([75628.471, 75628.521, 75628.571])

In [17]:
data.t[75628.521:75628.621].time


array([75628.521, 75628.571])